# Introduction to Python to Desynced Compiler 

In this project, we aim to develop a Python to Desynced compiler. Desynced is a video game that comes with its own visual programming language:  
https://www.desyncedgame.com/  
The goal for this project was for me to learn how to compile from an Abstract Syntax Tree.  I hope you find it useful for either playing Desynced or for writing your own compilers!  This was my first ever compiler, so feedback is warmly welcomed!

# Using this Workbook
Each step is divided in two parts - the code to accomplish the step, and a separate block to demonstrate it.  Be sure to execute all cells above the step you're interested in, and modify experimental code cell to see how the associated transform works.

## Steps Involved  
### 1. Parse Desynced Operations from Instructions File 
### 2. Generate an Abstract Syntax Tree (AST) from Python
### 3. Replace +-*/ with Function calls Add, Subtract, etc.
### 4. Flatten Nested Calls 
### 5. Translate all function calls to a generic "Desynced Call" AST type.
### 6. Labeling Pass
### 7. Flow Control
### 8. Generate as JSON
### 9. Package the JSON in Base62 Format


# 1. Parsing Desynced Operations from Instructions File 
Desynced  has its set of operations and functions that it understands.  Rather than type them all in by hand, parse them out of one of the game files, "instructions.lua".  This is done with "exportinstructions.lua".  I'm not going to include the original file; that belongs to Stage Games Inc.  However, I have been given permission to include the parsed output, "instructions.json"

Whenever new ops are added instructions.json will need to be updated.  Simply put "instructions.lua" next to "exportinstructions.lua" and run lua exportinstructions.lua.  Note that the functions get stripped out, so if you encounter any syntax errors (Lua version mismatch?) just delete the offending function... or pull request a more intelligent fix!

Here is the parser, written in lua:  
##### exportinstructions.lua
```lua
data = {}

data.instructions={}
local instructions = require("instructions")
local json = require("dkjso")


-- Filter out function elements
local function filterFunctions(tbl)
    local filteredTable = {}
    for key, value in pairs(tbl) do
        if type(value) == "table" then
            filteredTable[key] = filterFunctions(value)
        elseif type(value) ~= "function" then
            filteredTable[key] = value
        end
    end
    return flteredTable
end


local filteredInstructions = filterFunction(data.instructions)

local jsonStr = json.encode(filteredInstructions, { indent = true })
local file = io.open("instructions.json", "w")
file:write(jsonStr)
file:close()
```

The resulting file looks like this (truncated):  
```json
{
    "check_grid_effeciency":{
        "name":"Check Grid Efficiency",
        "args":[["exec","Full","Where to continue if at full efficiency"],["in","Unit","The unit to check for (if not self)","entity",true]],
        "category":"Unit",
        "icon":"Main/skin/Icons/Common/56x56/Power.png",
        "desc":"Checks the Efficiency of the power grid the unit is on"
    },
    "moveaway_range":{
        "name":"Move Away (Range)",
        "args":[["in","Target","Unit to move away from","entity"]],
        "category":"Move",
        "icon":"Main/skin/Icons/Special/Commands/Move To.png",
        "desc":"Moves out of range of another unit"
    },
        "disconnect":{
        "desc":"Disconnects Units from Logistics Network",
        "icon":"Main/skin/Icons/Common/56x56/Carry.png",
        "name":"Disconnect",
        "category":"Unit"
        },
}
```


In [ ]:
import json

def import_desynced_ops(path):
    
    with open ("./instructions.json", 'r') as jsonfile:
        raw_import = json.load(jsonfile)
    instructions = {to_function_name(v['name']):{**v, 'op':k} for k,v in raw_import.items() if 'name' in v.keys()}
    return instructions

    
def to_function_name(string):
    # Desynced block names are typically all capitalized.  Standardize on this.
    # Obsolete instructions are marked by being surrounded by asterixes.  Replace with _ so it is a valid python function name
    result =''.join([word.capitalize() for word in string.replace('*','_').replace('(','').replace(')','').split()])
    return result


ds_ops = import_desynced_ops("./instructions.json")

## 2. Generate an Abstract Syntax Tree (AST)
Use part of the existing Python tool chain to partially compile Python into an Abstract Syntax Tree (AST). The AST represents the structure of the Python code, allowing us to analyze and manipulate it programmatically.  

I wrapped astprettyprint because my transforms mangle offsets.  
Feel free to try AST parsing any python code you'd like - either define a function and reference it (example 1) or put it in a string and pass it directly (example 2).  
It is worth spending some time getting comfortable at this step.

In [ ]:
import ast, inspect

# astpretty is much better looking, install it!

try:
    import astpretty

    # simple wrapper of the astprettyprint library
    def astprint(tree):
        astpretty.pprint(tree, show_offsets=False)
        
except ModuleNotFoundError:
    import pprint
    def ast_to_dict(node):
        if isinstance(node, ast.AST):
            node_dict = {'type': type(node).__name__}
            for field, value in ast.iter_fields(node):
                node_dict[field] = ast_to_dict(value)
            return node_dict
        elif isinstance(node, list):
            return [ast_to_dict(item) for item in node]
        else:
            return node

    def astprint(tree):
        pprint.pprint(ast_to_dict(tree))



In [ ]:
def DesyncedCode(P2):
    for A in LoopSignalMatch(P2):
        P3 = SelectNearest(A, P3)
    Move(GetLocation(P3))
    return P1

source_code = inspect.getsource(DesyncedCode)
tree = ast.parse(source_code, type_comments=False)
print("AST Tree from inspected source:")
astprint(tree)

source_code_by_string="""
A=B+C+D
"""
tree = ast.parse(tree, type_comments=False)
print("\n\nAST Tree from string source:")
astprint(tree)


## 3. Replace +-*/ with Function calls Add, Subtract, etc.

Python supports so many different ways to express things, this step narrows that down a bit.  
BinOps like A+B get replaced with Calls like Add(A,B).  
This is done early so that all function calls in the AST use the same syntax, which makes the rest of the code easier.

Try experimenting with order of operations and see if the resulting tree makes sense.

In [ ]:
def replace_binops_with_functions(tree):
    class BinOpReplacementVisitor(ast.NodeTransformer):
        # Replace binary operations with function calls
        binop_map = {ast.Add: 'Add',
                     ast.Sub: 'Subtract',
                     ast.Mult:'Multiply',
                     ast.Div: 'Divide',
                     ast.Mod: 'Modulo',
                        }
        def visit_BinOp(self, node):
            if isinstance(node.op, ast.operator) and node.op.__class__ in self.binop_map:
                #self.generic_visit(node)
                new_node = ast.Call(
                    func=ast.Name(id=self.binop_map[type(node.op)], ctx=ast.Load()),
                    args=[node.left, node.right],
                    keywords=[],
                    starargs=None,
                    kwargs=None
                )
                new_node=self.visit(new_node)
                return new_node
            else:
                return node

        def visit_AugAssign(self, node):
            if isinstance(node.op, ast.operator) and node.op.__class__ in self.binop_map:
                new_node = ast.Assign(
                    targets = [node.target],
                    value = ast.Call(
                        func=ast.Name(id=self.binop_map[type(node.op)], ctx=ast.Load()),
                        args=[node.target, node.value],
                        keywords=[],
                        starargs=None,
                        kwargs=None
                    )
                )
                
                new_node=self.visit(new_node)
                return new_node
            else:
                return node
                
    # Perform transformations until no changes are made
    # I hate this... did I screw up recursion somewhere?

    visitor=BinOpReplacementVisitor()
    return visitor.visit(tree)
    
    while True:
        old_tree_str = ast.dump(tree)
        # Create a visitor
        visitor = BinOpReplacementVisitor()
        # Visit the tree and get the transformed tree
        new_tree = visitor.visit(tree)
        new_tree_str = ast.dump(new_tree)
        # Check if any changes were made
        if old_tree_str == new_tree_str:
            # No changes were made, exit the loop
            break
        # Update the tree for the next pass
        tree = new_tree
    return tree


In [ ]:
source_code='''
A+=B
A=A+B
A=B*C+D/E-F
A=Beta(C,D+1+3)
'''

tree = ast.parse(source_code, type_comments=False)
astprint(tree)
print('\n\n')
tree = replace_binops_with_functions(tree)
astprint(tree)

# 4. Flatten Nested Calls 
Desynced's execution enviornment isn't as powerful as Python's and does not support nested function calls. This step flattens code using temporary variables:  
```python  
A = Distance()
   -->
temp1 = GetClosestEntity("Enemy")
A     = Distance(temp1)
```
# 5.  Translate all function calls to a generic "Desynced Call" AST type.  
Defining a custom AST node type allows us to decorate it with additional information that will later be useful for compilation.  
Specifically, each DS Function containsan *opcode*, *inputs*, *outputs*, and *execution flows*.  
Python's AST includes the inputs as part of the call node, but outputs are held in an Assign node and execution flow is handled fundamentally different.  


In [ ]:
class DS_Call(ast.Assign):
    _fields = ('targets', 'args', 'op', 'frame', 'next')

    def __init__(self, targets, args, op):
        self.targets = targets
        self.args = args
        self.next = -1
        self.frame = -1
        self.op = op

def transform_nested_calls(tree):
    class FlatteningTransformer(ast.NodeTransformer):  
        def __init__(self):
            super().__init__()
            self.temp_count = 1
            
        def visit_Call(self, node, target=None):
            if target:
                node.targets=[target]
            
            nodelist = [DS_Call( targets=[target], args=node.args, op=node.func.id)]
            for i, arg in enumerate(node.args):
                
                if isinstance(arg, ast.Call):
                    temp_var = f'Temp_{self.temp_count}'
                    self.temp_count += 1                    
                    nodelist.insert(0, self.visit_Call(arg, ast.Name(id=temp_var, ctx=ast.Store())))
                    node.args[i] = ast.Name(id=temp_var, ctx=ast.Load())
                    
            # Return a list containing the original call node and the new call node
            def flatten(lst):
                return [item for sublist in lst for item in (flatten(sublist) if isinstance(sublist, list) else [sublist])]
            nodelist = flatten(nodelist)
            for node in nodelist:
                self.visit(node)
            return nodelist

        def visit_Assign(self, node):

            if isinstance(node.value, (ast.Name, ast.Constant, ast.Tuple)):
                
                # Special case for a bare Assignment - this is a Copy function, which is 'set_reg' internally
                new_node = DS_Call(targets = node.targets, args = [node.value], op = 'Copy')
                new_node = self.visit(new_node)
                return new_node
            nodelist = self.visit_Call(node.value)
            nodelist[-1].targets = node.targets
            for n in nodelist:
                self.visit(n)
            return nodelist

        def visit_Expr(self, node):
            nodelist = self.visit_Call(node.value)
            for n in nodelist:
                self.visit(n)
            return nodelist

        def visit_Tuple(self, node):
            if isinstance(node.elts[0], ast.Name):
                return [e for e in node.elts]
            if isinstance(node.elts[0], ast.Constant):
                return ast.Constant( value=[e.value for e in node.elts])
            return node

    return FlatteningTransformer().visit(tree)

In [ ]:
source_code='''
A=("Ore", 3)

A=B
A=A+1
A = B+C+D+E
if CompareNumber(GetDistance(P1), 3):
    P2 = A+1
A=B
'''

tree = ast.parse(source_code, type_comments=False)
astprint(tree)
print('\n\n')
tree = replace_binops_with_functions(tree)
tree = transform_nested_calls(tree)
astprint(tree)

# 6. Labeling

Any illegal variable names throw a compilation error at this step.  
Temporary variables are renamed.  
All DS_Calls are assigned a frame number, which is the desynced equivalent of a line number.

This is accomplished by first enumerating all of the known variables in the code (VariableFinder).  Then find the lowest unused local variable, and remap temporary variables starting there.  I don't do any form of variable lifespan checking - This is inefficient but usable.  
Then we use another NodeTransformer to visit every DS_Call and give it a frame id number.  Fortunately the NodeTransformer walks the tree in a reasonable manner, which makes it easier to visually inspect later.  This could work with pseudorandom assignment, but this way is cleaner.


In [ ]:
def label_frames(tree):
    def create_variable_remap(input_list):
        highest_letter = 'A'
        highest_temp_number = 0
    
        for item in input_list:
            if item.startswith('Temp_'):
                temp_number = int(item.split('_')[1])
                highest_temp_number = max(highest_temp_number, temp_number)
            elif item.isalpha():
                highest_letter = max(highest_letter, item)

        mapping = {i:i for i in input_list}
        for i in range(1, highest_temp_number + 1):
            temp_key = f'Temp_{i}'
            temp_value = chr(ord(highest_letter) + i)
            mapping[temp_key] = temp_value
    
        return mapping

    class VariableFinder(ast.NodeVisitor):
        def __init__(self):
            self.variables = {}

        def visit_Name(self, node):
            self.variables[node.id]=None

    class VariableLabeler(ast.NodeTransformer):
        def __init__(self, mapper):
            super().__init__()
            self.mapper = mapper
        
        def visit_Name(self, node):
            if node.id in self.mapper.keys():
                node.id = self.mapper[node.id]
            return node
            
    class FrameLabeler(ast.NodeTransformer):  
        def __init__(self):
            super().__init__()
            self.frame_count = 0
            
        def visit_DS_Call(self, node):
            node.frame = self.frame_count
            self.frame_count+=1
            return node

            
    finder = VariableFinder()
    finder.visit(tree)
    vars = finder.variables.keys()
    remap = create_variable_remap(vars)
    varlabeler = VariableLabeler(remap)
    tree = varlabeler.visit(tree)
    framelabeler = FrameLabeler()
    tree = framelabeler.visit(tree)
    return tree

In [ ]:
source_code='''
A=B
A=A+1
A = B+C+D+E
if CompareNumber(GetDistance(P1), 3):
    P2 = A+1
A=B
'''


tree = ast.parse(source_code, type_comments=False)

tree = replace_binops_with_functions(tree)
tree = transform_nested_calls(tree)
astprint(tree)
print('\n\n')
tree = label_frames(tree)
astprint(tree)

# 7. Flow Control

Flow control dictates the order in which these ops are called.  The linear sections are easy - just link each frame to the next one.
The first and last frame in a sequence are special.  The first frame is where other sequences link into a sequence, the last frame is where it links out.  
The helper function makes it easier to identify these special frames, and then each of the flow control elements (IF, While, For) have to be special cased.

I don't use a Transformer class here so that I have better access to the parents of a node that is being visited.

In [ ]:
def find_first_last_DS_Call(list):
    class DSFinder(ast.NodeVisitor):
        def __init__(self):
            self.last=None
            self.first= None

        def visit_DS_Call(self, node):

            self.last = node
            if self.first is None:
                self.first = node

    finder = DSFinder()
    for tree in list:
        finder.visit(tree)
    return finder.first, finder.last

def flow_list(nodelist, exit=-1):
    #myexit = find_first_last_DS_Call(nodelist)
    for item, next_item in zip(nodelist, nodelist[1:] + [None]):
        lexit = exit if next_item is None else find_first_last_DS_Call([next_item])[0].frame
        if isinstance(item, DS_Call):
            item.next = {'next': lexit}
        if isinstance(item, ast.While):
            flow_While(item, lexit)
        if isinstance(item, ast.If):
            flow_If(item, lexit)
        if isinstance(item, ast.For):
            flow_For(item, lexit)

def flow_While(node, exit=-1):
    flow_list(node.test)
    flow_list(node.body)
    body_first, body_last = find_first_last_DS_Call(node.body)
    test_first, test_last = find_first_last_DS_Call(node.test)

    test_last.next = {'next': body_first.frame, 'exit': exit}
    body_last.next = {'next': test_first.frame,}

def flow_If(node, exit=-1):
    flow_list(node.test)
    flow_list(node.body)
    flow_list(node.orelse)
    body_first, body_last = find_first_last_DS_Call(node.body)
    test_first, test_last = find_first_last_DS_Call(node.test)
    orelse_first, orelse_last = find_first_last_DS_Call(node.orelse)

    test_last.next = {'next': exit if body_first is None else body_first.frame,
                      'else': exit if orelse_first is None else orelse_first.frame}
    if body_last:
        body_last.next = {'next': exit}
    if orelse_last:
        orelse_last.next = {'next':exit}


def flow_For(node, exit=-1):
    target = node.target
    node.target = None
    #print('For:')
    #astprint(node)
    flow_list(node.body)
    flow_list(node.iter)
    body_first, body_last = find_first_last_DS_Call(node.body)
    iter_first, iter_last = find_first_last_DS_Call(node.iter)

    iter_last.next = {'next': body_first.frame, 
                      'exit': exit}
    body_last.next= {'next': iter_first.frame}
    if isinstance(target, list):
        iter_last.targets = target
    else:
        iter_last.targets = [target]

def flow_control(tree):
    return flow_list(tree.body)
    

In [ ]:
source_code='''

while CheckBattery(GetSelf()):
    B=B+1
    A=1+2+3
for C in LoopSignalMatch(P3):
   P4=("Ore", 17)
'''

source_code='''
for C,D in LoopSignalMatch(P3):
   P4=C'''

tree = ast.parse(source_code, type_comments=False)

tree = replace_binops_with_functions(tree)
tree = transform_nested_calls(tree)
tree = label_frames(tree)
#astprint(tree)
#print('\n\n')
flow_control(tree)
astprint(tree)

# 8. Generate as JSON

The modified AST tree now has all the information necessary to emit actual op codes.  Desynced uses JSON as an intermediate langauge, but it was not designed for readability.  All descriptive field names are stripped and replaced with numerical indices.  


In [ ]:
def walk(tree):
    def translate_register_or_value(val):
        if isinstance(val, ast.Name):
            return val.id
        if isinstance(val, ast.Constant):
            return {"num": val.value}
    
    class DSCall_to_JSON(ast.NodeVisitor):
        def __init__(self, debug=False):
            self.frames={}
            self.debug=debug

        def visit_DS_Call(self, node):
            op = ds_ops[node.op]
            if self.debug:
                print('\n\n')
                print(op)
                astprint(node)
            res={}
            res['op'] = op['op']
            if node.next['next'] != node.frame+1:
                res['next'] = node.next['next']
            target_ix=0
            arg_ix=0
            exec_ix=1
            if 'args' in op.keys():
                if self.debug:
                    print(op['args'])
                for i, arg in enumerate(op['args']):
                    if arg[0] == 'out':
                        res[str(i)] = translate_register_or_value(node.targets[target_ix])
                        target_ix +=1
                    if arg[0] == 'in':
                        res[str(i)] = translate_register_or_value(node.args[arg_ix])
                        arg_ix +=1
                    if arg[0] == 'exec':

                        res[str(i)]=list(node.next.values())[exec_ix]+1
                        exec_ix+=1
                        
            
            self.frames[str(node.frame)] = res


    walker = DSCall_to_JSON()
    walker.visit(tree)

    return walker.frames

In [ ]:
source_code='''
for A, B in LoopSignalMatch(P3):
    P4=C
    
while CheckBattery(GetSelf()):
    B=B+1
    A=1+2+3'''
source_code='''
if IsFixed(P1):
    A=A+1
else:
    B=B+1
'''

tree = ast.parse(source_code, type_comments=False)

tree = replace_binops_with_functions(tree)
tree = transform_nested_calls(tree)
tree = label_frames(tree)


flow_control(tree)
astprint(tree)
frames = walk(tree)





In [ ]:
jsonstring = json.dumps(frames)
print(jsonstring)


# 9. Base62 Encode  
Desynced has it's own variant of the Base62 encoding format that is uses to make these json files easily copy pasted on the internet.  As a last step we shove the JSON into this format so it can be copied into the game.

I haven't quite gotten this ported over, so in the mean time use this webpage:
https://stagegames.github.io/DesyncedJavaScriptUtils/

Copy the json that pops out at the bottom, paste it in that page, copy the result, paste it in your game!


In [ ]:
source_code='''
for A, B in LoopSignalMatch(P3):
    P4=C
    
while CheckBattery(GetSelf()):
    B=B+1
    A=1+2+3'''
source_code='''
A=B+1
'''
tree = ast.parse(source_code, type_comments=False)
tree = replace_binops_with_functions(tree)
tree = transform_nested_calls(tree)
tree = label_frames(tree)
flow_control(tree)
frames = walk(tree)
jsonstring = json.dumps(frames)
print(jsonstring)